In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn")
plt.rcParams["figure.figsize"] = 10, 6

In [2]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

In [308]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import log_loss, brier_score_loss, precision_score, recall_score, f1_score
from sklearn.decomposition import PCA

In [309]:
X = pd.read_csv("data/train.data.csv")
y = pd.read_csv("data/train.labels.csv")
test_data = pd.read_csv("data/test.data.csv")
X = preprocessing.scale(X.values)
y = y.values.ravel()
test_data = preprocessing.scale(test_data.values)

In [302]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [303]:
trainset = torch.utils.data.TensorDataset(torch.Tensor(X_train), torch.LongTensor(y_train))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=200,
                                          shuffle=True, num_workers=2)
testset = torch.utils.data.TensorDataset(torch.Tensor(X_test), torch.LongTensor(y_test))
testloader = torch.utils.data.DataLoader(testset, batch_size=200,
                                          shuffle=True, num_workers=2)

In [304]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(128, 500)
        self.fc2 = nn.Linear(500, 200)
        #self.fc3 = nn.Linear(500, 100)
        self.fc4 = nn.Linear(500, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        #x = self.fc2(x)
        #x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

net = Net()

In [305]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-4)

In [306]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

[1,    10] loss: 0.056
[1,    20] loss: 0.033
[1,    30] loss: 0.025
[1,    40] loss: 0.020
[1,    50] loss: 0.015
[1,    60] loss: 0.014
[2,    10] loss: 0.011
[2,    20] loss: 0.009
[2,    30] loss: 0.010
[2,    40] loss: 0.008
[2,    50] loss: 0.008
[2,    60] loss: 0.008
[3,    10] loss: 0.006
[3,    20] loss: 0.005
[3,    30] loss: 0.006
[3,    40] loss: 0.006
[3,    50] loss: 0.006
[3,    60] loss: 0.006
[4,    10] loss: 0.004
[4,    20] loss: 0.003
[4,    30] loss: 0.004
[4,    40] loss: 0.004
[4,    50] loss: 0.004
[4,    60] loss: 0.004
[5,    10] loss: 0.003
[5,    20] loss: 0.003
[5,    30] loss: 0.003
[5,    40] loss: 0.003
[5,    50] loss: 0.003
[5,    60] loss: 0.003
[6,    10] loss: 0.002
[6,    20] loss: 0.002
[6,    30] loss: 0.002
[6,    40] loss: 0.002
[6,    50] loss: 0.002
[6,    60] loss: 0.002
[7,    10] loss: 0.001
[7,    20] loss: 0.002
[7,    30] loss: 0.002
[7,    40] loss: 0.001
[7,    50] loss: 0.001
[7,    60] loss: 0.001
[8,    10] loss: 0.001
[8,    20] 

In [307]:
net.eval()
test_loss = 0
correct = 0
for data, target in testloader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = net(data)
    test_loss += F.cross_entropy(output, target, size_average=False).data[0] # sum up batch loss
    pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
    correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

test_loss /= len(testloader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(testloader.dataset),
    100. * correct / len(testloader.dataset)))


Test set: Average loss: 0.0794, Accuracy: 1466/1500 (98%)



In [360]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(128, 200)
        #self.do = nn.Dropout()
        #self.fc2 = nn.Linear(300, 200)
        #self.fc3 = nn.Linear(500, 100)
        self.fc4 = nn.Linear(200, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        #x = self.do(x)
        #x = self.fc2(x)
        #x = F.relu(x)
        #x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [361]:
sss = StratifiedShuffleSplit(n_splits=6, test_size=0.2)
for train, test in sss.split(X, y):
    X_train = X[train,:]
    y_train = y[train]
    X_test = X[test,:]
    y_test = y[test]
    
    trainset = torch.utils.data.TensorDataset(torch.Tensor(X_train), torch.LongTensor(y_train))
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=300, shuffle=True, num_workers=2)
    testset = torch.utils.data.TensorDataset(torch.Tensor(X_test), torch.LongTensor(y_test))
    testloader = torch.utils.data.DataLoader(testset, batch_size=300, shuffle=True, num_workers=2)
    
    net = Net()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.002, weight_decay=2e-4)
    
    for epoch in range(10):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs
            inputs, labels = data

            # wrap them in Variable
            inputs, labels = Variable(inputs), Variable(labels)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            #running_loss = loss.data[0]
            #print('[%d] loss: %.3f' % (epoch + 1, running_loss))

    net.eval()
    test_loss = 0
    correct = 0
    for data, target in testloader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = net(data)
        test_loss += F.cross_entropy(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(testloader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
            test_loss, correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))

Test set: Average loss: 0.1062, Accuracy: 2885/3000 (96%)
Test set: Average loss: 0.1103, Accuracy: 2881/3000 (96%)
Test set: Average loss: 0.1045, Accuracy: 2884/3000 (96%)
Test set: Average loss: 0.1046, Accuracy: 2881/3000 (96%)
Test set: Average loss: 0.1049, Accuracy: 2887/3000 (96%)
Test set: Average loss: 0.1117, Accuracy: 2884/3000 (96%)
